In [2]:
%load_ext autoreload
%autoreload 2

In [19]:
import os,sys,inspect
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import datetime
import re
from dateutil.relativedelta import relativedelta
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
data_dir = os.path.join(parentdir, 'data')
job_alpha = os.path.join(parentdir, 'src')


In [28]:
search = os.path.join(data_dir, 'sample-search.htm')
with open(search, 'r') as myfile:
    data=myfile.read().replace('\n', '')

def get_past_date(str_days_ago):
    TODAY = datetime.date.today()
    splitted = str_days_ago.split()
    if len(splitted) == 1 and splitted[0].lower() == 'today':
        return str(TODAY.isoformat())
    elif len(splitted) == 1 and splitted[0].lower() == 'yesterday':
        date = TODAY - relativedelta(days=1)
        return str(date.isoformat())
    elif splitted[1].lower() in ['hour', 'hours', 'hr', 'hrs', 'h']:
        date = datetime.datetime.now() - relativedelta(hours=int(splitted[0]))
        return str(date.date().isoformat())
    elif splitted[1].lower() in ['day', 'days', 'd']:
        date = TODAY - relativedelta(days=int(splitted[0]))
        return str(date.isoformat())
    elif splitted[1].lower() in ['wk', 'wks', 'week', 'weeks', 'w']:
        date = TODAY - relativedelta(weeks=int(splitted[0]))
        return str(date.isoformat())
    elif splitted[1].lower() in ['mon', 'mons', 'month', 'months', 'm']:
        date = TODAY - relativedelta(months=int(splitted[0]))
        return str(date.isoformat())
    elif splitted[1].lower() in ['yrs', 'yr', 'years', 'year', 'y']:
        date = TODAY - relativedelta(years=int(splitted[0]))
        return str(date.isoformat())
    else:
        return "Wrong Argument format"
    
soup = BeautifulSoup(data, 'html.parser')
jobids = []
titles = []
companies = []
locations = []
areas = []
category = []
subcategory = []
headers = []
short_descriptions = []
dates = []
salaries = []
for i, article in enumerate(soup.find_all('article')):
    jobids.append(None)
    titles.append(None)
    companies.append(None)
    locations.append(None)
    areas.append(None)
    category.append(None)
    subcategory.append(None)
    headers.append(None)
    short_descriptions.append(None)
    today = datetime.date.today()
    dates.append(today.strftime('%Y-%m-%d'))
    salaries.append(None)
    
    jobids.insert(i, article['data-job-id'])
    for title in article.select('h1 a'):
        titles.insert(i, title.string)
    for item in article.find_all('a', attrs={'data-automation' : True}):
        if item['data-automation'] == 'jobCompany':
            companies.insert(i, item.string)
        if item['data-automation'] == 'jobLocation':
            locations.insert(i, item.string) 
        if item['data-automation'] == 'jobArea':
            areas.insert(i, item.string) 
        if item['data-automation'] == 'jobClassification':
            category.insert(i, item.string) 
        if item['data-automation'] == 'jobSubClassification':
            subcategory.insert(i, item.string)
    for header in article.find_all('ul'):
        headers.insert(i, '. '.join([header.string for header in article.select('li span span')]))
    for item in article.find_all('span', attrs={'data-automation' : True}):
        if item['data-automation'] == 'jobListingDate':
            date = get_past_date(' '.join(re.split('(\d+)', item.string.replace('ago', ''))))
            dates.insert(i, date)
        if item['data-automation'] == 'jobShortDescription':
            short_descriptions.insert(i, item.string)
        if item['data-automation'] == 'jobSalary':
            salaries.insert(i, item.string)

results = list(zip(jobids, dates, titles, companies, locations, areas, category, subcategory, salaries, headers, short_descriptions))
df = pd.DataFrame(results, columns=['jobid', 'date', 'title', 'company', 'location', 'area', 'category', 'subcategory', 'salary', 'header', 'shortdescription'])
df['tick'] = 'bhp'
df['description'] = None
df

,jobid,date,title,company,location,area,category,subcategory,salary,header,shortdescription,tick,description
0,37264209,2018-09-24,Superintendent Mining Production | South Flank,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,"Mining, Resources & Energy",Management,None,Flexible work arrangements available. Leadersh...,Join our South Flank leadership team to help d...,bhp,None
1,37264245,2018-09-24,Superintendent Mining Production | South Flank,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,"Mining, Resources & Energy",Mining - Operations,None,Flexible work arrangements available. Leadersh...,Join our South Flank leadership team to help d...,bhp,None
2,37292687,2018-09-22,Production Technician | Mount Whaleback | Resi...,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,"Mining, Resources & Energy",Mining - Operations,None,None,We have a number of residential opportunities ...,bhp,None
3,37292491,2018-09-22,"Principal Infrastructure Architect (Perth, Bri...",BHP Billiton Group,Perth,"CBD, Inner & Western Suburbs",Information & Communication Technology,Architects,None,None,the Principal Technical Architect will have re...,bhp,None
4,37292267,2018-09-22,Advanced Riggers,Western Work Force Pty Ltd,"Port Hedland, Karratha & Pilbara",None,Trades & Services,Other,None,Various Positions Available. Immediate Starts....,We are currently seeking Advanced Riggers for ...,bhp,None
5,37292111,2018-09-22,Mechanical Fitters | Shutdowns,Western Work Force Pty Ltd,"Port Hedland, Karratha & Pilbara",None,Trades & Services,"Fitters, Turners & Machinists",None,Excellent Pay Rates . Immediate Start . FIFO,We have immediate Shutdown opportunities for e...,bhp,None
6,37291818,2018-09-22,Mechanical Engineer - Technical Standards | Ya...,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,"Mining, Resources & Energy",Mining - Engineering & Maintenance,None,None,Provide specialised engineering technical supp...,bhp,None
7,37291222,2018-09-22,"Logistics Administrator (Newcastle, NSW)",BHP Billiton Group,"Newcastle, Maitland & Hunter",None,"Manufacturing, Transport & Logistics","Production, Planning & Scheduling",None,None,Looking to kick start your career in Mining Lo...,bhp,None
8,37290165,2018-09-22,Supervisor Maintenance,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,"Mining, Resources & Energy",Other,None,None,"Do you consider yourself to be motivated, with...",bhp,None
9,37290166,2018-09-22,Supervisor Maintenance,BHP Billiton Group,"Port Hedland, Karratha & Pilbara",None,Engineering,Maintenance,None,None,"Do you consider yourself to be motivated, with...",bhp,None


In [26]:
details = os.path.join(data_dir, 'sample-details.htm')
with open(details, 'r') as myfile:
    details_data=myfile.read().replace('\n', '')
soup = BeautifulSoup(details_data, 'html.parser')
descr = soup.find('div', attrs={'data-automation': 'jobDescription'})
job_details = descr.get_text()
job_details = re.sub(r'\s+', ' ', job_details).strip().lower()
job_details

'we are looking for full-time and casual sales consultant the applicants must be: experienced in telecommunications sales,customer service and motivated to meet or exceed ongoing store sales and customer expectations. confident in siebel and mnc systems (will be preferred)! self-motivated,team player and have an ability to learn quickly. confident with excellent communication skills, listening skills and able to build great rapport with customers. able to deliver high level of pre and post sales advocacy (nps) strong in organisational and task management skills with attention to detail. professional at all times with excellent personal presentation and people skills. able to work under pressure and deliver daily results. mature with positive and "can do "attitude and a high level of initiative very reliable, punctual, trustworthy and hardworking with good work ethics able to attend day starts, sales and product training sessions as required. able to work weekends and extra hours as and